In [47]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

### Checking is GPU connection available

In [48]:
print(torch.backends.mps.is_available())

True


### Connecting to a device's GPU (on mac it is 'mps')

In [49]:
device = torch.device('cpu')
print(device)

cpu


In [50]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

In [51]:
train_set = torchvision.datasets.MNIST(
    root='data/',
    train=True,
    download= True,
    transform=transforms
)

In [52]:
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

In [53]:
misol = next(iter(train_data))
misol[0].shape

torch.Size([64, 1, 28, 28])

In [54]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()


        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

             nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(32*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        return self.net(x)

In [55]:
model = SimpleCNN().to(device)

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr = 0.001)

epoches = 10

losses = []

for epoch in range(epoches):

    model.train()
    total_losses = 0

    for data, label in train_data:
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output, label)

        loss.backward()
        optimizer.step()

        total_losses += loss

    print(f'Epoxa : {epoch}, Loss : {total_losses / len(train_data)}\n\n')

Epoxa : 0, Loss : 0.20100966095924377


Epoxa : 1, Loss : 0.05505594238638878


Epoxa : 2, Loss : 0.03959861025214195


Epoxa : 3, Loss : 0.028963439166545868


Epoxa : 4, Loss : 0.02318253368139267


Epoxa : 5, Loss : 0.01842491514980793


Epoxa : 6, Loss : 0.014693639241158962


Epoxa : 7, Loss : 0.01229526661336422


Epoxa : 8, Loss : 0.010948248207569122


Epoxa : 9, Loss : 0.009233430027961731




In [57]:
torch.save(model.state_dict(), "simplecnn_state.pth")